# Random Forest Regressor (Multioutput) - Evaluation of phosphoproteomic markers as potential drug response predictors

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


In [ ]:
# loading phosphoproteomic data (top 12 sites) and drug response data with responses to 318 drugs

X_phospho = pd.read_csv('phosphproteomics_top_12.csv').set_index('sites')

y_responses = pd.read_csv('dss_imputed_318.csv')


In [ ]:
# transposing the DataFrame to have patients as rows and features (sites) as columns

X_phospho = X_phospho.T


In [ ]:
# scaling due to significantly different ranges

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_phospho)


### Running RF regressor - hyperparameter tuning

In [ ]:
# splitting datasets to train and test (80/20)

X_train, X_test, y_train, y_test = train_test_split(X_phospho, y_responses, test_size=0.2, random_state=42)


In [ ]:
# defining the model

rfr = RandomForestRegressor()


In [ ]:
# defining the hyperparameter grid for the RandomForestRegressor

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 2, 3],
    'min_samples_split': [2,3,4],
}

# setting up the grid search for the RandomForestRegressor CV

grid_search = GridSearchCV(rfr, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)


### Predicting 

In [ ]:
# getting the best model and wrapping it in MultiOutputRegressor

best_model = MultiOutputRegressor(grid_search.best_estimator_)

# making predictions

y_pred = best_model.fit(X_train, y_train).predict(X_test)


### Ploting drugs with responses predicted well by 12 phosphosites

In [ ]:
# converting y_test and y_pred to numpy arrays

y_test_np = np.array(y_test)
y_pred_np = np.array(y_pred)

# getting the number of drugs
num_drugs = y_test_np.shape[1]

# getting the drug names from y_responses column names
drug_names = list(y_responses.columns)

# setting the number of rows and columns for subplots (3 plots per row)
num_plots_per_row = 3
num_rows = int(np.ceil(num_drugs / num_plots_per_row))
num_cols = num_plots_per_row

# creating subplots with smaller size
fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 4*num_rows)) 

# flatten axes for easier indexing
axes = axes.flatten()

# plotting actual versus predicted values for each drug with R-squared > 0.6

plot_count = 0

for i in range(num_drugs):
    
    # extracting actual and predicted values for the current drug
    actual_values = y_test_np[:, i]
    predicted_values = y_pred_np[:, i]
    
    # calculating the R-squared value
    r_squared = r2_score(actual_values, predicted_values)
    
    # getting the drug name from y_responses column names
    drug_name = drug_names[i]
    
    # plot only if R-squared is above 0.6
    if r_squared > 0.6:
        axes[plot_count].scatter(actual_values, predicted_values, color='blue', alpha=0.5)
        axes[plot_count].plot([min(actual_values), max(actual_values)], [min(actual_values), max(actual_values)], color='red', linestyle='--')
        axes[plot_count].set_title(f"{drug_name}\nR^2 = {r_squared:.2f}", fontsize=12)
        axes[plot_count].set_xlabel("Actual Values", fontsize=10)
        axes[plot_count].set_ylabel("Predicted Values", fontsize=10)
        axes[plot_count].tick_params(axis='both', which='major', labelsize=8)
        axes[plot_count].grid(True)
        plot_count += 1

# hide empty subplots
for j in range(plot_count, num_rows * num_cols):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()